In [1]:
from data_utils import *
import pandas as pd

In [2]:
data = get_CUSTOMDATA()

In [11]:
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']

print(X_train.shape)

(652845, 8)


In [12]:
print(X_train[0])

[-1.95450681 12.05935099  5.18643323 -0.93489879  1.11204038 -1.93099587
  0.07171381 -1.89690202]


In [31]:
y_train = data['y_train']
y_train = np.argmax(y_train, axis=1)

y_val = data['y_val']
y_val = np.argmax(y_val, axis=1)

y_test = data['y_test']
y_test = np.argmax(y_test, axis=1)

print(y_train)

[3 5 5 ... 2 5 5]


In [32]:
print(y_train.shape)

(652845,)


In [36]:
train_X_df = pd.DataFrame(X_train, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
train_y_df = pd.DataFrame(y_train, columns=['target'])

train_df = pd.concat([train_X_df, train_y_df], axis=1)

val_X_df = pd.DataFrame(X_val, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
val_y_df = pd.DataFrame(y_val, columns=['target'])

val_df = pd.concat([val_X_df, val_y_df], axis=1)

test_X_df = pd.DataFrame(X_test, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
test_y_df = pd.DataFrame(y_test, columns=['target'])

test_df = pd.concat([test_X_df, test_y_df], axis=1)

print(train_df.head())
print(val_df.head())
print(test_df.head())


      emg_0      emg_1      emg_2      emg_3     emg_4      emg_5     emg_6  \
0 -1.954507  12.059351   5.186433  -0.934899   1.11204  -1.930996  0.071714   
1  0.045493  14.059351   0.186433  19.065101   4.11204  -1.930996 -0.928286   
2  0.045493  -0.940649   0.186433   1.065101  39.11204   5.069004  2.071714   
3 -1.954507  -4.940649  34.186433   7.065101   2.11204 -17.930996  0.071714   
4 -2.954507   5.059351  -1.813567   0.065101   1.11204   3.069004  0.071714   

      emg_7  target  
0 -1.896902       3  
1  1.103098       5  
2  4.103098       5  
3  1.103098       5  
4  2.103098       5  
      emg_0      emg_1      emg_2     emg_3    emg_4     emg_5      emg_6  \
0 -7.954507   4.059351  12.186433 -8.934899 -2.88796 -0.930996  -0.928286   
1  1.045493 -38.940649 -19.813567  6.065101  3.11204  2.069004   2.071714   
2  1.045493   0.059351   8.186433 -0.934899  0.11204  1.069004  -1.928286   
3  3.045493  22.059351   4.186433  4.065101  4.11204 -0.930996 -13.928286   
4 -0.954

In [37]:
train_df.to_hdf('../data/custom_train.hdf5', key='df', mode='w')
val_df.to_hdf('../data/custom_val.hdf5', key='df', mode='w')
test_df.to_hdf('../data/custom_test.hdf5', key='df', mode='w')

In [38]:
# Load them back and check
train_check = pd.read_hdf('../data/custom_train.hdf5', key='df')
val_check = pd.read_hdf('../data/custom_val.hdf5', key='df')
test_check = pd.read_hdf('../data/custom_test.hdf5', key='df')

print(f"Loaded train shape: {train_check.shape}")
print(f"Loaded val shape: {val_check.shape}")
print(f"Loaded test shape: {test_check.shape}")

Loaded train shape: (652845, 9)
Loaded val shape: (72534, 9)
Loaded test shape: (363250, 9)
